# CHIPPR

This notebook demonstrates the use of the Cosmological Hierarchical Inference with Probabilistic Photometric Redshifts (CHIPPR) package to estimate the redshift density function.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import timeit
import cProfile, pstats, StringIO
import os

import chippr
from chippr import *

In [ ]:
#help(chippr)

## Simulation

In [ ]:
tru_amps = np.array([0.20, 0.35, 0.55])
tru_means = np.array([0.5, 0.2, 0.75])
tru_sigmas = np.array([0.4, 0.2, 0.1])

tru_nz = gmix(tru_amps, tru_means, tru_sigmas, limits=(0., 1.))

In [ ]:
N =10**4 #2**16

tru_zs = tru_nz.sample(N)

# plt.hist(tru_zs, bins=50, normed=1)
# plt.xlabel(r'true $z$')
# plt.ylabel(r'probability density')

In [ ]:
params = 'params.txt'
params = sim_utils.ingest(params)
print(params)

In [ ]:
bin_ends = np.array([0., 1.])
weights = np.array([1.])

int_prior = discrete(bin_ends, weights)

In [ ]:
pr = cProfile.Profile()
pr.enable()

results_loc = os.path.join(os.path.join('..', 'research'), 'results')

posteriors = catalog(params, results_loc)
output = posteriors.create(tru_zs, int_prior)

data = np.exp(output['log_interim_posteriors'])

pr.disable()
s = StringIO.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print s.getvalue()

In [ ]:
plt.hist(posteriors.obs_samps, bins=100, normed=True, color="k")
plt.plot(posteriors.x_coarse, tru_nz.evaluate(posteriors.x_coarse), "r-")
plt.plot(posteriors.x_coarse, np.sum(data, axis=0) / N, "go")
plt.xlabel("z")

In [ ]:
for n, z in enumerate(data[:10]):
    plt.plot(posteriors.x_coarse, data[n], 'ko')
    plt.plot(posteriors.x_fine, posteriors.obs_lfs[n], 'k-')
    plt.show()

In [ ]:
saved_location = 'data.txt'
posteriors.write(saved_location)

## Inference

In [ ]:
saved_location = 'data.txt'
simulated_posteriors = catalog()
data = simulated_posteriors.read(saved_location)

In [ ]:
zs = data['bin_ends']
nz_intp = np.exp(data['log_interim_prior'])
z_posts = np.exp(data['log_interim_posteriors'])

z_difs = zs[1:]-zs[:-1]
z_mids = (zs[1:]+zs[:-1])/2.
n_bins = len(z_mids)

In [ ]:
# prior_sigma = 0.16
# prior_var = np.eye(n_bins)
# for b in range(n_bins):
#     prior_var[b] = 1. * np.exp(-0.5 * (z_mids[b] - z_mids) ** 2 / prior_sigma ** 2)
# l = 1.e-4
# prior_var = prior_var+l*np.identity(n_bins)

prior_var = np.eye(n_bins)
for k in range(n_bins):
    prior_var[k] = 1. * np.exp(-0.5 * (z_mids[k] - z_mids) ** 2 / 0.05 ** 2)

prior_mean = nz_intp
prior = mvn(prior_mean, prior_var)

In [ ]:
nz = log_z_dens(data, prior, truth=tru_nz, vb=True)

In [ ]:
nz_stacked = nz.calculate_stacked()

In [ ]:
nz_mmap = nz.calculate_mmap()

In [ ]:
nz_mexp = nz.calculate_mexp()

In [ ]:
nz_stats = nz.compare()

In [ ]:
pr = cProfile.Profile()
pr.enable()

nz_mmle = nz.calculate_mmle(nz_stacked)

pr.disable()
s = StringIO.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print s.getvalue()

In [ ]:
n_ivals = 2*n_bins
initial_values = prior.sample(n_ivals)

pr = cProfile.Profile()
pr.enable()

nz_samps = nz.calculate_samples(initial_values)

pr.disable()
s = StringIO.StringIO()
sortby = 'tottime'
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats()
print s.getvalue()

In [ ]:
nz.info['estimators'].keys()

In [ ]:
nz.plot_estimators('final_plot.png')

In [ ]:
nz.write('nz.p')

In [ ]:
nz.info = nz.read('nz.p')
print(nz)